In [15]:
import os
import numpy as np
import librosa
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import joblib

# -----------------------------
# CONFIG
# -----------------------------
N_MFCC = 13          # 13 is classical for HMMs
MAX_PAD_LEN = 174    # keep same as your pipeline
N_STATES = 8         # hidden states per HMM (tuneable)
COV_TYPE = "diag"    # covariance type for GaussianHMM
N_ITER = 300         # EM iterations for each HMM
MODEL_PATH = "hmm_only_emotion_models.joblib"

# -----------------------------
# FEATURE EXTRACTION
# -----------------------------
def extract_features(file_path, max_pad_len=MAX_PAD_LEN, n_mfcc=N_MFCC):
    signal, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)  # shape (n_mfcc, frames)
    # transpose to (frames, n_mfcc)
    mfcc = mfcc.T
    if mfcc.shape[0] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    else:
        mfcc = mfcc[:max_pad_len, :]
    return mfcc  # shape (time, n_mfcc)

# -----------------------------
# DATASET PATHS (edit to your dataset)
# -----------------------------
data_dirs = [
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\voice_data\\train_data\\SER",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_01",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_02",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_03",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_04",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_05",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_06",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_07",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_08",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_09",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_10",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_11",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_12",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_13",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_14",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_15",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_16",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_17",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_18",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_19",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_20",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_21",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_22",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_23",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_24",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Angry",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Happy",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Natural",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Sad",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Surprised"
]

# -----------------------------
# LOAD DATA (MFCC sequences) AND LABELS
# -----------------------------
X, y, emotions = [], [], []
for data_dir in data_dirs:
    if not os.path.exists(data_dir):
        print(f"⚠️ Warning: {data_dir} not found, skipping...")
        continue
    for fname in os.listdir(data_dir):
        if not fname.lower().endswith(".wav"):
            continue
        label = fname.split("_")[-1].replace(".wav", "")
        if label not in emotions:
            emotions.append(label)
        path = os.path.join(data_dir, fname)
        mfcc_seq = extract_features(path)         # shape (T, n_mfcc)
        X.append(mfcc_seq)
        y.append(emotions.index(label))

if len(X) == 0:
    raise RuntimeError("❌ No audio data found. Check your data_dirs paths.")

X = np.array(X, dtype=object)
y = np.array(y)

# -----------------------------
# TRAIN / TEST SPLIT
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# -----------------------------
# TRAIN HMMs (one per emotion)
# -----------------------------
choice = input("Do you want to load the saved model? (yes/no): ").strip().lower()

models = {}  # dict: emotion -> trained GaussianHMM

if choice == "yes" and os.path.exists(MODEL_PATH):
    print("🔁 Loading saved HMM models...")
    saved = joblib.load(MODEL_PATH)
    models = saved["models"]
    emotions = saved["emotions"]
    print(f"Loaded {len(models)} HMMs: {list(models.keys())}")
else:
    print("🧠 Training HMMs for each emotion...")
    for i, emotion in enumerate(emotions):
        # gather all sequences of this emotion
        seqs = [X_train[j] for j in range(len(X_train)) if y_train[j] == i]
        if len(seqs) == 0:
            print(f"⚠️ No training samples for emotion '{emotion}', skipping.")
            continue

        # Concatenate for hmmlearn: X_concat shape (N_total_frames, n_mfcc)
        lengths = [s.shape[0] for s in seqs]
        X_concat = np.vstack(seqs).astype(np.float64)

        model = hmm.GaussianHMM(n_components=N_STATES, covariance_type=COV_TYPE,
                                n_iter=N_ITER, verbose=True, random_state=42)

        # Fit
        print(f"Training HMM for emotion '{emotion}' on {len(seqs)} sequences ({X_concat.shape[0]} frames)...")
        model.fit(X_concat, lengths)
        models[emotion] = model
        print(f"✅ Trained HMM for '{emotion}'")

    # Save trained models and emotions
    joblib.dump({"models": models, "emotions": emotions,
                 "n_mfcc": N_MFCC, "max_pad_len": MAX_PAD_LEN}, MODEL_PATH)
    print(f"💾 Saved HMM models to {MODEL_PATH}")

# -----------------------------
# PREDICTION UTILITIES
# -----------------------------
def predict_emotion(file_path):
    """
    Given a wav file path, extract MFCC (T, n_mfcc), then compute the log-likelihood
    for each emotion HMM and return the emotion with highest score.
    """
    mfcc = extract_features(file_path)  # shape (T, n_mfcc)
    best_emotion = None
    best_score = -np.inf
    for emotion, model in models.items():
        try:
            score = model.score(mfcc)  # log-likelihood
        except Exception as e:
            score = -np.inf
        if score > best_score:
            best_score = score
            best_emotion = emotion
    return best_emotion

# -----------------------------
# EVALUATE ON TEST SET
# -----------------------------
print("🧪 Evaluating on test set...")
y_pred = []
for seq in tqdm(X_test):
    # each seq is (T, n_mfcc)
    best_emotion = None
    best_score = -np.inf
    for emotion, model in models.items():
        try:
            score = model.score(seq)
        except Exception:
            score = -np.inf
        if score > best_score:
            best_score = score
            best_emotion = emotion
    if best_emotion is None:
        y_pred.append(-1)
    else:
        y_pred.append(emotions.index(best_emotion))

valid_idx = [i for i, v in enumerate(y_pred) if v != -1]
if len(valid_idx) == 0:
    print("⚠️ No valid predictions were made on test set.")
else:
    acc = accuracy_score(y_test[valid_idx], np.array(y_pred)[valid_idx])
    print(f"✅ HMM Test Accuracy: {acc*100:.2f}% on {len(valid_idx)} sequences")

# -----------------------------
# SINGLE FILE PREDICTION TEST
# -----------------------------
test_file = "K:\\Code\\Project\\Research Paper\\Emotion Detection\\voice_data\\test_data\\F_01_OISHI_S_2_SURPRISE_2.wav"
if os.path.exists(test_file):
    print("Predicted Emotion:", predict_emotion(test_file))
else:
    print("⚠️ Test file not found:", test_file)


🧠 Training HMMs for each emotion...
Training HMM for emotion 'angry' on 685 sequences (119190 frames)...


         1 -6926125.06177819             +nan
         2 -5435567.87623626 +1490557.18554194
         3 -2055157.99378820 +3380409.88244806
         4 -706350.65489308 +1348807.33889511
         5 -628027.29357147  +78323.36132161
         6 -575854.75758605  +52172.53598542
         7 -549319.58463193  +26535.17295412
         8 -530209.80802395  +19109.77660798
         9 -509887.43501334  +20322.37301062
        10 -490653.24292061  +19234.19209273
        11 -465036.13745827  +25617.10546234
        12 -432743.17210645  +32292.96535182
        13 -391358.64968001  +41384.52242644
        14 -342188.09240812  +49170.55727189
        15 -285650.26730426  +56537.82510386
        16 -219455.61148135  +66194.65582291
        17 -126979.07346917  +92476.53801218
        18   -7417.54243128 +119561.53103789
        19   81970.26974299  +89387.81217427
        20   94440.41538637  +12470.14564338
        21   96025.36412334   +1584.94873697
        22   96977.35121581    +951.98709247
    

✅ Trained HMM for 'angry'
Training HMM for emotion 'disgust' on 506 sequences (88044 frames)...


         1 -5093089.37862890             +nan
         2 -4607925.07548246 +485164.30314644
         3 -3878287.30886119 +729637.76662127
         4 -1598612.91346946 +2279674.39539173
         5 -1498645.13570926  +99967.77776020
         6 -1454397.40180638  +44247.73390289
         7 -1410487.76860667  +43909.63319970
         8 -1389091.32862364  +21396.43998303
         9 -1376053.56186201  +13037.76676163
        10 -1364435.28325367  +11618.27860834
        11 -1352606.84140337  +11828.44185030
        12 -1340213.39479654  +12393.44660683
        13 -1326997.14702179  +13216.24777475
        14 -1312924.14875487  +14072.99826692
        15 -1298574.33861677  +14349.81013810
        16 -1282692.25020140  +15882.08841537
        17 -1264705.86021672  +17986.38998468
        18 -1246263.02750844  +18442.83270828
        19 -1221620.33583176  +24642.69167668
        20 -1188906.46164324  +32713.87418852
        21 -1147889.22897207  +41017.23267117
        22 -1113496.01075831  +34

✅ Trained HMM for 'disgust'
Training HMM for emotion 'fear' on 495 sequences (86130 frames)...


...

✅ Trained HMM for 'sad'
Training HMM for emotion 'calm' on 158 sequences (27492 frames)...


         2 -1422173.73860085 +129452.28128406
         3 -1212904.09768432 +209269.64091653
         4 -1203492.12290739   +9411.97477693
         5 -1199203.25982797   +4288.86307942
         6 -1195560.64237921   +3642.61744876
         7 -1190018.99144307   +5541.65093614
         8 -1179196.50826801  +10822.48317507
         9 -1157593.35009990  +21603.15816811
        10 -1140812.06503167  +16781.28506823
        11 -1134888.38232994   +5923.68270173
        12 -1130534.41647666   +4353.96585328
        13 -1125536.18446879   +4998.23200787
        14 -1119285.61485452   +6250.56961427
        15 -1112533.82383442   +6751.79102010
        16 -1103905.79194916   +8628.03188526
        17 -1091820.84020724  +12084.95174193
        18 -1072254.90014421  +19565.94006303
        19 -1043838.84217380  +28416.05797041
        20 -1016903.00414791  +26935.83802589
        21 -985414.14863127  +31488.85551664
        22 -952848.28071459  +32565.86791668
        23 -928070.07197489  +24778.

✅ Trained HMM for 'calm'
💾 Saved HMM models to hmm_only_emotion_models.joblib
🧪 Evaluating on test set...


100%|██████████| 798/798 [00:01<00:00, 559.26it/s]

✅ HMM Test Accuracy: 38.97% on 798 sequences
Predicted Emotion: angry
